In [ ]:
import osmnx as ox
import momepy
import geopandas as gpd
import shapely
from shapely.geometry import LineString
import os

In [ ]:
local_crs = 3414

place = "Singapore"

latlng = (1.29, 103.85)
dist = 30000

study_area = gpd.read_file("./source/Singapore_studyArea.shp").to_crs(epsg=4326)

# assuming your geodataframe is called `gdf` and the geometry column is called `geometry`
study_area_polygon = study_area.geometry.unary_union

In [ ]:
## Use osmnx to download all water body polygons within study_area
## and prune them from study_area
    # Download water body polygons
water = ox.geometries.geometries_from_polygon(study_area_polygon, tags={"natural": "water"})
water.head()

In [ ]:
## Prune study area
study_area = study_area.overlay(water, how='difference')

In [ ]:
study_area.plot()

In [ ]:
# buildings = ox.geometries.geometries_from_place(place, tags={'building':True})
# buildings = ox.geometries.geometries_from_point(latlng, tags={'building':True}, dist=dist)
buildings = ox.geometries.geometries_from_polygon(study_area_polygon, tags={'building':True})
buildings.head()

In [ ]:
buildings = buildings[buildings.geom_type == "Polygon"].reset_index(drop=True)
buildings = buildings[["geometry"]].to_crs(local_crs)
print(buildings.geom_type.value_counts())


In [ ]:
# Use the unary_union method to merge adjacent or overlapping polygons
merged = buildings.geometry.unary_union

# Convert the merged geometry back to a GeoDataFrame
merged_buildings_gdf = gpd.GeoDataFrame(geometry=[merged])
buildings = merged_buildings_gdf.explode()
buildings = buildings[buildings.geom_type == "Polygon"].reset_index(drop=True)

In [ ]:
buildings["uID"] = range(len(buildings))

In [ ]:
print(buildings.geom_type.value_counts())
buildings.head()

In [ ]:
osm_graph= ox.graph.graph_from_polygon(study_area_polygon, network_type='drive')
osm_graph = ox.projection.project_graph(osm_graph, to_crs=local_crs)
streets = ox.graph_to_gdfs(
    osm_graph,
    nodes=False,
    edges=True,
    node_geometry=False,
    fill_edge_geometry=True
)

streets.head()
streets = streets.drop(columns=[col for col in streets.columns if col != 'geometry'])

In [ ]:
from shapely.geometry import LineString

# Assuming your GeoDataFrame is named 'gdf'
line_strings = []
for geometry in streets.geometry:
    if geometry.geom_type == 'LineString':
        line_strings.append(geometry)
    elif geometry.geom_type == 'MultiLineString':
        line_strings.extend(list(geometry))

In [ ]:
collection = shapely.GeometryCollection(line_strings)  # combine to a single object
noded = shapely.node(collection)  # add missing nodes
streets_noded_gdf = gpd.GeoDataFrame(geometry=gpd.GeoSeries(noded), crs=local_crs)


In [ ]:
streets_noded_gdf.plot()

In [ ]:
buildings.plot()

In [ ]:
study_area = study_area.to_crs(local_crs)
study_area.plot()

In [ ]:
## create directory ./out/singapore if it does not exist
def create_dir(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
        
create_dir("./out/singapore")

In [ ]:
## convert streets_noded_gdf, buildings, and study_area to local_crs
streets_noded_gdf.to_parquet("./out/singapore/streets.pq")

buildings.to_parquet("./out/singapore/buildings.pq")

study_area.to_parquet("./out/singapore/study_area.pq")

In [25]:
water.drop(columns=water.columns.difference(['geometry'])).to_crs(local_crs).reset_index(drop=True).to_parquet("./out/singapore/water.pq")

In [ ]:
# # Create a polygon geodataframe from creating a 2 meter buffer around every line in streets_noded_gdf and dissolve it into study_area

# streets_noded_gdf_buffer = dgpd.from_geopandas(streets_noded_gdf, npartitions=4)
# streets_noded_gdf_buffer.buffer(2)

# study_area_dgpd = dgpd.from_geopandas(study_area, npartitions=4).append(streets_noded_gdf_buffer).dissolve()

In [ ]:
# test = study_area_dgpd.compute()

In [ ]:
# # dissolve streets_noded_gdf_buffer into study_area into one multipolygon in a geodataframe
# concat = pd.concat([study_area, streets_noded_gdf_buffer])

# study_area_polygon = gdf.geometry.unary_union
# study_area = gpd.GeoDataFrame(geometry=[dissolved_geom], crs=gdf.crs)

In [ ]:
# enclosures = momepy.enclosures(noded_gdf , limit= study_area.to_crs(local_crs))

In [ ]:
# enclosures.plot()

In [ ]:
# # Perform a spatial join of the overlapping polygons with themselves
# spatial_join = gpd.sjoin(enclosures, enclosures, how="inner", op="intersects")

# # Count the number of overlapping polygons for each polygon
# overlapping_counts = spatial_join.groupby(["eID_left"]).size()

# # Get the polygons that overlap with more than one other polygon
# overlapping_count = overlapping_counts[overlapping_counts > 30].index.tolist()
